# Module 4: NGS data quality control

## Overview

A typical whole genome sequencing process involves sample preparation, library preparation and sequencing. Errors occurring at each of these steps can negatively impact the quality of the sequence information. For example, the samples could be mixed during sample or library preparation, or errors can be encountered during the sequencing itself. If these errors are not removed from the raw reads, they might be incorporated into your analysis output and would be harder to resolve later on. Therefore, it is important to perform quality checks on the raw sequence reads before starting your analysis.

Here are some of the parameters for NGS data quality control.

- Check for primers and adaptors
- Trim low quality ends
- Remove low quality reads
- Remove short sequences
- Remove reads with a high proportion of ambiguous bases (“N”)
- Remove duplicates

There are several bioinformatic tools available for evaluating read data quality, here we will discuss one of the widely used tools, called [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) which is made available by the Babraham Institute.



### Install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Install software

In [ ]:
# Install FastQC and trimmomatic
!conda install -c bioconda fastqc -y
!conda install -c bioconda trimmomatic -y

### Download data

In [ ]:
!wget https://zenodo.org/records/14231070/files/Module_4.tar.gz

### Extract the .tar.gz file 

In [ ]:
!tar xvf Module_4.tar.gz

We will navigate to the folder containing the paired FASTQ files ERR2667694_1.fastq.gz and ERR2667694_2.fastq.gz, which correspond to the run accession ERR2667694 from the project [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084).

Some important data about the sample:

- Country of origin: Peru
- Organism: *Streptococcus pneumoniae*
- Instrument Platform: ILLUMINA
- Instrument Model: HiSeq X Ten
- Read Count: 3418378
- Base Count: 516175078
- Center Name: Wellcome Sanger Institute; SC
- Library Layaout: PAIRED
- Library strategy: WGS

We will navigate to the folder containing the fastq files

In [ ]:
# change directory
%cd Module_4/

In [ ]:
# list all files
!ls

Execute FastQC

In [ ]:
# Run fastqc
!fastqc *.fastqc

You will have the following output:

![qc](images/fastqc1.png)

The QC folder will now have the following files:

![qc](images/fastqc2.png)

After checking all the parameters, QC report will be generated for the reads in **ERR2667694_1_fastqc.html** and  **ERR2667694_2_fastqc.html**  files.  

"Download the .html files to your computer. On the left side of the Colab screen, you will find a folder icon, select it and locate the files. Select the file and click 'Download':

![qc](images/fastqc3.png)

**Let’s explore the  ERR2667694_1_fastqc.html!**

### 01. Basic Statistics


The Basic Statistics module generates some simple composition statistics for the file analysed. 

- **Filename**: The original filename of the file which was analysed

- **File type**: Says whether the file appeared to contain actual base calls or colorspace data which had to be converted to base calls.

- **Encoding**: Says which ASCII encoding of quality values was found in this file

- **Total Sequences**: A count of the total number of sequences processed. There are two values reported, actual and estimated. At the moment these will always be the same. In the future it may be possible to analyse just a subset of sequences and estimate the total number, to speed up the analysis, but since we have found that problematic sequences are not evenly distributed through a file we have disabled this for now. 

- S**equence Length**: Provides the length of the shortest and longest sequence in the set. If all sequences are the same length only one value is reported. 

- **%GC**: The overall %GC of all bases in all sequences

![qc](images/QC1.png)


### 02. Per base sequence quality 

Scrolling down the HTML file is the ‘per base sequence quality’ figure, shown above. The graph represents the average quality of bases over the entire read length. The y-axis on the graph shows the quality scores. The space coloured in the green regions shows high quality, the amber coloured region below reflects acceptable quality and the regions in the red shows low quality. Therefore, if you observe the line in the red region for your sequence reads, it means the probability of a base call being incorrect is higher.

For each position a BoxWhisker type plot is drawn. The elements of the plot are as follows:

- The central red line is the median value
- The yellow box represents the inter-quartile range (25 - 75)
- The upper and lower whiskers represent the 10
- The blue line represents the mean quality

The quality of calls on most platforms will degrade as the run progresses, so it is common to see base calls falling into the orange areas towards the end of a read, 

It should be mentioned that there are a number of different ways to encode a quality score in a FastQ file. FastQC attempts to automatically determine which encoding method was used, but in some very limited datasets it is possible that it will guess this incorrectly (ironically only when your data is universally very good!). The title of the graph will describe the encoding FatsQC thinks your file used.

![qc](images/QC2.png)

### 03. Per tile sequence quality

This graph will only appear in your analysis results if you’re using an illumina library which retains its original sequence identifiers. Encoded in these is the flowcell tile from which each read came. The graph allows you to look at the quality scores from each tile across all of your bases to see if there was a loss in quality associated with only one part of the flow cell. 

The plot shows the deviation from the average quality for each tile. The colours are on a cold to hot scale, with cold colours being positions where the quality was at or above the average for that base in the run, and hotter colours indicate that a tile had worse qualities than other tiles for that base. A good plot should be blue all over.

Reasons for seeing warnings or errors on this plot could be transient problems such as bubbles going through the flowcell, or they could be more permanent problems such as smudges on the flowcell or debris inside the flowcell lane. 

![qc](images/QC3.png)

### 04. Per Sequence Quality Scores

The per sequence quality score report allows you to see if a subset of your sequences have universally low quality values. It is often the case that a subset of sequences will have universally poor quality, often because they are poorly imaged (on the edge of the field of view etc), however these should represent only a small percentage of the total sequences. 

If a significant proportion of the sequences in a run have overall low quality then this could indicate some kind of systematics problem - possible with just part of the run (for example one end of a flowcell).

![qc](images/QC4.png)

### 05. Per Base Sequence Content

Per Base Sequence Content plots out the proportion of each base position in a file for which each of the four normal DNA bases has been called. 

In a random library you would expect that there would be little to no difference between the different bases of a sequence run, so the lines in this plot should run parallel with each other, The relative amount of each base should reflect the overall amount of these bases in your genome, but in any case they should not be hugely imbalanced from each other.

It’s worth noting that some types of library will always produce biased sequence composition, normally at the start of the read. Libraries produced by priming using random hexamers (including nearly all RNA-seq libraries) and those which were fragmented using transposases inherit an intrinsic bias in the positions at which reads start. This bias does not concern an absolute sequence, but instead provides enrichment of a number of different k-mers at the 5’end of the reads. Whilst this is a true technical bias, it isn't something which can be corrected by trimming and in most cases does not seem to adversely affect the downstream analysis. It will however produce a warning or error in this module. 

![qc](images/QC5.png)

### 06. Per Sequence GC Content

This module measures the GC content across the whole length of each sequence in a file and compares it to a modelled normal distribution of GC content.

In a normal random library you would expect to see a roughly normal distribution of GC content  where the central peak corresponds to the overall GC content of the underlying genome. Since we don't know the GC content of the genome the modal GC content miscalculated from the observed data and used to build a reference distribution.

An unusually shaped distribution could indicate a contaminated library or some other kind of biased subset. A normal distribution which is shifted indicates some systematic bias which is independent of base position. If there is a systematics bias which creates a shifted normal distribution then this won't be flagged as an error by the module since it doesn't know what your genome’s GC content.

![qc](images/QC6.png)

### 07. Per Base N Content

If a sequencer is unable to make a base call with sufficient confidence then it will normally substitute an N rather than a conventional base call. 

This module plots out the percentage of base calls at each position for which an N was called. 

It's not unusual to see a very low proportion of Ns appearing in a sequence especially nearer the end of a sequence. However, if this proportion rises above a few percent it suggests that the analysis pipeline was unable to interpret the data well enough to make valid base calls. 

![qc](images/QC7.png)

### 08. Sequence Length Distribution

Some high throughput sequencers generate fragments of uniform length but others can contain reads of wildly varying lengths. Even with uniform length libraries, some pipelines will trim sequences to remove poor quality base calls from the end. 

This module generates a graph showing the distribution of fragment sizes in the file which was analysed.

In many cases this will produce a simple graph showing a peak only at one size, but for variable length FastQ files this will show the relative amounts of each different size of sequence fragment. 

![qc](images/QC8.png)

### 09. Sequence Duplication Levels

In a diverse library most sequences will occur only once in the final set. A low level of duplicating may indicate a very high level of coverage of the target sequence, but a high level of duplication is more likely to indicate some kind of enrichment bias (e.g., PCR over amplification).

This module counts the degree of duplication for every sequence in a library and creates a plot showing the relative number of sequences with different degrees of duplications.

To cut down on the memory requirements for this module only sequences which first appear in the first 100,000 sequences in each file are analysed, but this should be enough to get a good impression for the duplication levels in the whole file. Each sequence is tracked to the end of the file to give a representative count of the overall duplication level. To cut down on the amount of information in the final plot any sequence with more than 10 duplicates are placed into grouped bins to give a clear impression of the overall duplication level without having to show each individual duplication value. 

Because the duplication detection requires an exact sequence match over the whole length of the sequence, any reads over 75bps in length are truncated to 50bp for the purposes of this analysis. Even so, longer reads are more likely to contain sequencing errors which will artificially increase the observed diversity and will tend to underrepresent highly duplicated sequences.

The plot shows the proportion of the library which is made up of sequences in each of the different duplication level bins. There are two lines in the plot. The  blue line takes the full sequence set and shows how its duplication levels are distributed. In the red plot, the sequences are de-duplicated and the proportions shown are the proportions of the de-duplicated set which come from different duplication levels in the original data. 

In a properly diverse library, most sequences should fall into the far left of the plot in both the red and blue limes. A general level of enrichment, indicating broad over sequencing in the library will tend to flatten the lines, lowering the low end and generally raising other categories. More specific enrichments of subsets, or the presence of low complexity contaminants will tend to produce spikes towards the right of the plot. These high duplication peaks will most often appear on the blue trace as they make up a high proportion of the original library, but usually disappear in the read trace as they make up an insignificant proportion of the deduplicated set. If peaks persist in the blue trace, then this suggests that there are a large number of different highly duplicated sequences which might indicate either a contaminant set or a very severe technical duplication. 

The module also calculates an expected overall loss of sequence were the library to be duplicated. This headline figure is shown at the top of the plot and gives a reasonable impression of the potential overall level of loss.

![qc](images/QC9.png)

### 10. Overrepresented sequences

A normal high-throughput library will contain a diverse set of sequences, with no individual sequence making up a tiny fraction of the whole. Finding that a single sequence is very over-represented in the set either means that it is highly biologically significant, or indicates that the library is contaminated , or not as diverse as you expected. 

This module lists all of the sequences which make up more than 0.1. 

For each overrepresented sequence the program will look for matches in a database of common contaminants and will report the hit it finds. Hits must be at least 20bp in length and have no more than 1 mismatch. Finding a hit doesn't necessarily mean that this is the source of the contamination, but may point you in the right direction. It's also worth pointing out that many adapter sequences are very similar to each other so you may get a hit reported which isn't technically correct, but which has a very similar sequence to the actual match. 

![qc](images/QC10.png)

### 11. Adapter Content

The k-mer content module will do a generic analysis of all of the k-mers in your library to find those which do not have even coverage through the length of your reads. This can find a number of different sources of bias in the library which can include the presence of read-through adapter sequences building up  the end of your sequences.

You can however find that the presence of any overrepresented sequences in your library (such as adapter dimers) will cause the k-mer plot to be dominated by the k-mers these sequences contain, and that it's not always easy to see if there are other biases present in which you might be interested.

One obvious class of sequences which you might want to analyse are adapter sequences. It is useful to know if your library contains a significant amount of adapter in order to be able to assess whether you need to trim the adapter or not. Although the k-mer analysis can theoretically spot this kind of contamination it isn't always clear. This module therefore does a specific search for a set of separately defined k-mers and will give you a view of the total proportion of your library which contains these k-mers. A results trace will always be generated for all the sequences present in the adapter config file so you can see the adapter content of your library, even if it is low. 

The plot itself shows a cumulative percentage count of the proportion of your library which has seen each of the adapter sequences at each position. Once a sequence has been seen in a read, it is counted as being present right through to the end of the read so the percentages you see will only increase as the read length goes on. 

![qc](images/QC11.png)


Here is the summary of all the statistics of all the statistics according to developer’s page: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

___

# Adapter Trimming

## Overview

Trimming of adapter sequences from short read data is a common preprocessing step during NGS data analysis. When performing paired-end sequencing, the overlap between forward and reverse read can be used to identify excess adapter sequences.

Illumina FASTQ file generation pipelines include an adapter trimming option for the removal of adapter sequences from the 3’ ends of reads. Adapter sequences should be removed from reads because they interfere with downstream analyses, such as alignment of reads to a reference. The adapters contain the sequencing primer binding sites, the index sequences, and the sites that allow library fragments to attach to the flow cell lawn. Libraries prepared with Illumina library prep kits require adapter trimming only on the 3’ ends of reads, because adapter sequences are not found on the 5’ ends.

Further reading

1. https://support.illumina.com/bulletins/2016/04/adapter-trimming-why-are-adapter-sequences-trimmed-from-only-the--ends-of-reads.html 

2. https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1069-7


We will assess the quality of **spneumo_R1.fastq.gz** and **spneumo_R2.fastq.gz** in the Adapter_trimming folder.

In [ ]:
# move to the adapter trimming directory
%cd Adapter_trimming

In [ ]:
# run fastqc
!fastqc spneumo_R1.fastq.gz spneumo_R2.fastq.gz

Download the HTML file at the left panel and open it in your browser. You will see the quality of the reads.

![example](images/example.png))

We use **trimmomatic** tool to remove adaptors, to trim low quality reads and to remove short sequences. 

To execute the trimmomatic tool, we will run the command: 

In [ ]:
!trimmomatic PE spneumo_R1.fastq.gz spneumo_R2.fastq.gz spneumo_R1.trimmed.fastq.gz  /dev/null spneumo_R2.trimmed.fastq.gz /dev/null ILLUMINACLIP:./adapters/TruSeq3-PE.fa:2:30:10 SLIDINGWINDOW:4:20 MINLEN:36

An explanation of this command is as follows:

- **trimmomatic**: is the tool

- **PE**: That it will be taking paired end file as input

- **spneumo_R1.fastq.gz**: The first input file name

- **spneumo_R2.fastq.gz**: The second input file name

- **spneumo_R1.trimmed.fastq.gz**: The output file for surviving pairs from the _1 file

- **/dev/null**: Discards the output file for orphaned reads from the _1 file 

- **spneumo_R1.trimmed.fastq.gz**: The output file for surviving pairs from the _2 file

- **/dev/null**: Discards the output file for orphaned reads from the _2 file

- **ILLUMINACLIP**:adapters/TruSeq3-PE.fa:2:30:10: To clip the illumina adapters from the input file using the adapter sequences listed in TruSeq3-PE.fa. The numbers 2:30:10 tell trimmomatic how to handle sequence matches to the TruSeq3 adapters.

- **SLIDINGWINDOW:4:20**:  To use a sliding window of size 4 that will remove bases if their phred score is below 20

- **MINLEN:36**: This will discard and reads that do not have a at least 36 bases remaining after this trimming step

You will have the following output:

![trimming](images/trimming.png)

Now run fastqc on the trimmomatic output files *.trimmed.fastq.gz using the command:

In [ ]:
# Run fastqc on the trimmed files
!fastqc *.trimmed.fastq.gz

### Questions


> What percent of reads did you discard from your sample?

> What percent of reads did you keep?

> How different is the HTML report for the spneumo_R1.trimmed.fastq.gz/spneumo_R2.trimmed.fastq.gz  from the spneumo_R1.fastq.gz/spneumo_R2.fastq.gz?

## BONUS!

If you are working with BASH in your computer or in a HPC and you have too many files you can optimize commands, loops are very useful for large datasets.

Here's a way to do it. 

Create a new bash script using nano named `trimming.sh`

In [ ]:
# Dont run this cell
# create a script to run the trimming
!nano trimming.sh

Then copy and paste the following scrip in your new file:

In [ ]:
#!/bin/bash
#Author: Nathalia Portilla

for i in $(ls *_1.trimmed.fastq.gz); do

NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"

trimmomatic PE $j $k ${NAME}_1.trimmed.fastq.gz /dev/null ${NAME}_2.trimmed.fastq.gz /dev/null ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 SLIDINGWINDOW:6:30 MINLEN:50

done

Save the file.

Finally, you can execute it:

In [ ]:
# Dont run this cell
!bash trimming.sh

*Adapted from:*

- Advanced Bioinformatics Course developed for the GPS and JUNO projects - Wellcome Sanger Insitute

*Modified by Luisa Sacristán (Universidad de los Andes-CABANA)*